<a href="https://colab.research.google.com/github/MinhNguyen2000/TME_6015/blob/main/Assignment_1/TME6015_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 1. Import libraries

In [9]:
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.preprocessing import image
import numpy as np
import os
from PIL import Image

In [10]:
# Load the MNIST dataset
(x_train, y_train), (x_test, y_test) = mnist.load_data()